In [10]:
import os
import pandas as pd
import numpy as np
import json
import sys

In [11]:
# Get the current working directory
CURRENT_DIRECTORY = os.getcwd()
# Get the parent directory
PARENT_DIRECTORY = os.path.dirname(CURRENT_DIRECTORY)
PARENT_DIRECTORY = os.path.dirname(PARENT_DIRECTORY)

sys.path.append(PARENT_DIRECTORY + '//config/')
from config import count_sequences_above_threshold, count_sequences_below_threshold, normalize_sensor_data

# Open the config file and load its content into a dictionary
config_file = open(PARENT_DIRECTORY + '\\config\\config.json')
CONFIG_DATA = json.load(config_file)

# Close the file after loading the data
config_file.close()

In [12]:
NORMALIZE_SENSOR_DATA = CONFIG_DATA['NORMALIZE_SENSOR_DATA']
WINDOW_SIZE = CONFIG_DATA['WINDOW_SIZE']
NUMBER_OF_FEATURES = 8

CLASSIFICATIONS = CONFIG_DATA['CLASSES_MOTIONTYPE']

LABEL_COLUMN = 'Class_MotionType'

# processed data folder path
DATA_FOLDER = PARENT_DIRECTORY + '\\processed-training-data\\4-PROCESSED-DATA\TRAIN2\\'

# save numpy data folder
NUMPY_DATA_FOLDER_FILE_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\'

# v0 data (not normalized, use to generate more data)
ALL_X_TRAIN_CLASSIFY_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\training\\ALL-X-TRAIN-CLASSIFY-MOTIONTYPE-LSIDESTEPS-V0.npy'
ALL_Y_TRAIN_CLASSIFY_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\training\\ALL-Y-TRAIN-CLASSIFY-MOTIONTYPE-LSIDESTEPS-V0.npy'

In [13]:
LABEL_TO_CATEGORY = {label: category for category, label in enumerate(CLASSIFICATIONS)}

print(LABEL_TO_CATEGORY)

{'LAR': 0, 'SML': 1}


In [14]:
file_names = [file for file in os.listdir(DATA_FOLDER) if file.endswith('.xlsx') and "SIDESTEPS-L-" in file and os.path.isfile(os.path.join(DATA_FOLDER, file))]

print(file_names)

['PROC-TRAIN2-SIDESTEPS-L-LAR-110BPM.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-LAR-120BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-LAR-160BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-LAR-164BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-LAR-220BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-LAR-30BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-LAR-40BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-LAR-41BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-LAR-55BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-LAR-60BPM.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-LAR-80BPM.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-LAR-82BPM.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-SML-120BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-SML-160BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-SML-30BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-SML-40BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-SML-60BPM.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-SML-80BPM.xlsx']


In [15]:
def df2Xy(df, windowSize=5):
  X = []
  y = []

  # NORMALIZE THE SENSOR DATA
  if NORMALIZE_SENSOR_DATA == True:
    df["L_Pitch"] = df["L_Pitch"].apply(normalize_sensor_data)
    df["L_Roll"] = df["L_Roll"].apply(normalize_sensor_data)
    df["R_Pitch"] = df["R_Pitch"].apply(normalize_sensor_data)
    df["R_Roll"] = df["R_Roll"].apply(normalize_sensor_data)
    df["L_Pitch_Delta"] = df["L_Pitch_Delta"].apply(normalize_sensor_data)
    df["L_Roll_Delta"] = df["L_Roll_Delta"].apply(normalize_sensor_data)
    df["R_Pitch_Delta"] = df["R_Pitch_Delta"].apply(normalize_sensor_data)
    df["R_Roll_Delta"] = df["R_Roll_Delta"].apply(normalize_sensor_data)

  for i in range( len(df) - windowSize + 1):
    # inputs: X rows
    # form a new input which has size of our windowSize
    input_data_list_readings = []

    # loop through each row in our windowsize
    for j in range(windowSize):
        # fetch sensor data for this row
        row_values_readings = df.loc[i + j, ['L_Pitch', 'L_Roll', 'R_Pitch', 'R_Roll']].values.tolist()

        # add row values to the input
        input_data_list_readings.append(row_values_readings)

    # turnn list into array to do arthimetic
    raw_sensor_data = np.array(input_data_list_readings)

    min_sensor_reading = np.min(raw_sensor_data, axis=0)
    max_sensor_reading = np.max(raw_sensor_data, axis=0)

    input_data_array = np.concatenate((min_sensor_reading, max_sensor_reading))

    if(np.isnan(input_data_array).any() == False):
      # turn back to list
      input_data_list = input_data_array.tolist()

      # add our input to our total inputs, marked as X
      X.append(input_data_list)

      # outputs: y labels
      label = df.loc[i + (windowSize - 1), [LABEL_COLUMN]].values.tolist()

      y.append(label)

  return (np.array(X), np.array(y))

In [16]:
ALL_X_TRAIN = np.empty((0, NUMBER_OF_FEATURES))  # List to store all X training data
ALL_Y_TRAIN = np.empty((0))  # List to store all Y training data

In [17]:
def processData(ALL_X, ALL_Y):
    for fileName in file_names:
        # Read the Excel file
        df = pd.read_excel(DATA_FOLDER + fileName)

        xTrain, yTrain = df2Xy(df, WINDOW_SIZE)

        yTrain = yTrain.reshape(-1)
        
        ALL_X = np.concatenate([ALL_X, xTrain])
        ALL_Y = np.concatenate([ALL_Y, yTrain])
        
        print('xTrain.shape:', xTrain.shape)
        print('ALL_X_TRAIN.shape:', ALL_X.shape, ' ALL_Y_TRAIN.shape:', ALL_Y.shape)
        print('-------------------------------')

    # Create the folder if it doesn't exist
    if not os.path.exists(NUMPY_DATA_FOLDER_FILE_PATH):
        os.makedirs(NUMPY_DATA_FOLDER_FILE_PATH)

    # convert to numerical labels (originally text labels)
    numerical_label = np.vectorize(LABEL_TO_CATEGORY.get)(ALL_Y)

    # Assuming your numpy array is called 'data_array'
    np.save(ALL_X_TRAIN_CLASSIFY_PATH, ALL_X)
    np.save(ALL_Y_TRAIN_CLASSIFY_PATH, numerical_label)

In [18]:
processData(ALL_X_TRAIN, ALL_Y_TRAIN)

ALL_X_TRAIN_CLASSIFY = np.load(ALL_X_TRAIN_CLASSIFY_PATH)
ALL_Y_TRAIN_CLASSIFY = np.load(ALL_Y_TRAIN_CLASSIFY_PATH, allow_pickle=True)

print(ALL_X_TRAIN_CLASSIFY.shape)
print(ALL_Y_TRAIN_CLASSIFY.shape)

xTrain.shape: (2285, 8)
ALL_X_TRAIN.shape: (2285, 8)  ALL_Y_TRAIN.shape: (2285,)
-------------------------------
xTrain.shape: (4078, 8)
ALL_X_TRAIN.shape: (6363, 8)  ALL_Y_TRAIN.shape: (6363,)
-------------------------------
xTrain.shape: (3795, 8)
ALL_X_TRAIN.shape: (10158, 8)  ALL_Y_TRAIN.shape: (10158,)
-------------------------------
xTrain.shape: (3135, 8)
ALL_X_TRAIN.shape: (13293, 8)  ALL_Y_TRAIN.shape: (13293,)
-------------------------------
xTrain.shape: (2285, 8)
ALL_X_TRAIN.shape: (15578, 8)  ALL_Y_TRAIN.shape: (15578,)
-------------------------------
xTrain.shape: (8175, 8)
ALL_X_TRAIN.shape: (23753, 8)  ALL_Y_TRAIN.shape: (23753,)
-------------------------------
xTrain.shape: (7609, 8)
ALL_X_TRAIN.shape: (31362, 8)  ALL_Y_TRAIN.shape: (31362,)
-------------------------------
xTrain.shape: (6289, 8)
ALL_X_TRAIN.shape: (37651, 8)  ALL_Y_TRAIN.shape: (37651,)
-------------------------------
xTrain.shape: (4588, 8)
ALL_X_TRAIN.shape: (42239, 8)  ALL_Y_TRAIN.shape: (42239,)
-

In [19]:
print(np.max(ALL_X_TRAIN_CLASSIFY))

37.0
